## CUSP London Data Dive 2019, London Ambulance Service Data, Group A 

Research question: Is travel distance to place of work associated with reason for leaving?

In [ ]:
import pandas as pd
import geopandas as gpd
import shapely
from matplotlib import pyplot as plt
import numpy as np
import pysal as ps
import pygeoprocessing
import sklearn

%matplotlib inline

In [ ]:
datadir = '/Users/lucieburgess/Documents/CUSP_data_dive_2019/data/Workforce_Data/'
leavers = pd.read_csv(datadir + 'LAS_Leavers_070319.csv')
staff = pd.read_csv(datadir + 'LAS_Staff_in_Post_070319.csv')
work_locations = pd.read_csv(datadir + 'workplace_location_table.csv')


In [ ]:
leavers.head()

In [ ]:
leavers.groupby('Job Role').describe()

In [ ]:
df = leavers[['Job Role','Employee Number']].groupby(['Job Role'])['Employee Number'].size().reset_index(name='count').sort_values('count', ascending=False)
print(df)

In [ ]:
staff.columns

In [ ]:
staff.head()

In [ ]:
# Postcode translated to a lat, long co-ordinate and a easting and northing using an online tool
work_locations.columns

In [ ]:
work_locations.head()

In [ ]:
work_locations = work_locations.drop(['manual'], axis=1)

In [ ]:
# Link the locations to the workforce data and convert to a geopandas dataframe
# rename 'esrLocationFull' in work_locations and 'Location' in staff to 'ambulance_station_location'

work_locations.rename(columns={'esrLocationFull':'ambulance_station_location'}, inplace=True)
staff.rename(columns={'Location' : 'ambulance_station_location'},inplace = True)

staff = pd.merge(staff, work_locations, how='inner', on=['ambulance_station_location'])

In [ ]:
staff.columns

In [ ]:
# Convert Easting and Northing to int as currently strings
# NB. astype(np.int64) doesn't work because of NaNs.

staff['work_location_easting'] = pd.to_numeric(staff['work_location_easting'], errors='coerce')
staff['work_location_northing'] = pd.to_numeric(staff['work_location_northing'], errors='coerce')

In [ ]:
staff['work_coords'] = list(zip(staff.work_location_easting, staff.work_location_northing))

In [ ]:
staff['work_coords'] = staff['work_coords'].apply(Point)

In [ ]:
# geo-coded home postcode data, collected using the Google API:
work_home = pd.read_csv(datadir + 'work_home_cor.csv')

In [ ]:
work_home.head()

In [ ]:
work_home = work_home.drop(['Unnamed: 0'], axis=1)

In [ ]:
staff=pd.merge(staff,work_home, how = 'inner', on = 'IDnumber')

In [ ]:
staff.columns

In [ ]:
staff.shape #5763 rows, 39 columns

In [ ]:
# Convert home_lon and home_lat to co-ordinates and then easting, northing
staff['home_coords'] = list(zip(staff.home_lon, staff.home_lat))
staff['home_coords'] = staff['home_coords'].apply(Point)

In [ ]:
# Deal with NaNs
# Drop the rows with null StartLatitude or null StartLongitude so we don't have Point(nan nan) in the geodataframe
staff = staff.loc[(staff.home_lon.notnull() & staff.home_lat.notnull())]
staff.shape # 5756,40

In [ ]:
# Add the geometry
staff_geo = gpd.GeoDataFrame(staff, geometry='home_coords', crs={'init': 'epsg:4326'})

In [ ]:
# transform the CRS to BNG co-ordinates
staff_geo = staff_geo.to_crs({'init':'epsg:27700'})

In [ ]:
staff_geo.geometry.name

In [ ]:
staff_geo.head()

In [ ]:
staff_geo.plot() # now this is in the correct CRS - British National Grid

In [ ]:
# Add LSOA level data for Greater London
shape_dir = "/Users/lucieburgess/Documents/KCL/Urban_Mind_data/statistical-gis-boundaries-london/ESRI/" 
lsoas = gpd.read_file(shape_dir + "LSOA_2011_London_gen_MHW.shp")

lsoas.plot()

In [ ]:
# Check the co-ordinate reference system of the LSOA data.
lsoas.crs # already in BNG co-ordinates, OSGB36 which is what we want.

In [ ]:
# Spatial join of the LSOA to workforce_geo so we have the LSOA of the home postcode of the member of staff.
# Match each home point in workforce_geo to an LSOA. 
staff_geo_lsoa = gpd.sjoin(staff_geo, lsoas, how="inner", op='within')

In [ ]:
staff_geo_lsoa.shape # 3633 staff - a significant number live outside London!

In [ ]:
# Look at data of LSOA for England and Wales
shape_dir = '/Users/lucieburgess/Documents/CUSP_data_dive_2019/lsoa_2011/'
lsoas_ew = gpd.read_file(shape_dir + 'Lower_Layer_Super_Output_Areas_December_2011_Generalised_Clipped__Boundaries_in_England_and_Wales.shp')
lsoas_ew.plot()

In [ ]:
# Now join the data again to the England and Wales LSOA dataset
# This seems to have lost the spatial information in the LSOA geometry column
# From the GeoPandas documentation: ‘inner’: use intersection of keys from both dfs; retain only left_df geometry column
staff_geo_lsoa_ew = gpd.sjoin(staff_geo, lsoas_ew, how="inner", op='within')

In [ ]:
staff_geo_lsoa_ew.columns

In [ ]:
staff_geo_lsoa_ew.geometry.name

In [ ]:
# re-set the geometry to 'geometry'
#staff_geo_lsoa_ew_2.set_geometry('geometry')

In [ ]:
staff_geo_lsoa_ew.shape # 5751, 47

In [ ]:
# How to mask the areas in which there is no LSOA data?
# Run regression or random forest of the three variables that are common between the leavers and staff datasets

In [ ]:
staff_geo_lsoa_ew = staff_geo_lsoa_ew.drop(['index_right'], axis = 1)

In [ ]:
# join the health subdomain IMD data on LSOA code
imd_dir = '/Users/lucieburgess/Documents/CUSP_data_dive_2019/imd/' 
imd = pd.read_csv(imd_dir + 'File_2_ID_2015_Domains_of_deprivation.csv', header='infer', dtype={'Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived)': np.float64,
                                   'Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs)': np.float64})

In [ ]:
imd.columns

In [ ]:
lsoas_ew.columns
imd.columns
imd.rename(columns={'LSOA code (2011)':'lsoa11cd'}, inplace=True)
imd_lsoa_ew = lsoas_ew.merge(imd, on='lsoa11cd')

In [ ]:
imd_lsoa_ew.columns
type(imd_lsoa_ew)
imd_lsoa_ew.geometry.name
imd_lsoa_ew.crs

In [ ]:
staff_geo_lsoa_ew = staff_geo_lsoa_ew.drop(['objectid'], axis=1)

In [ ]:
staff_geo_lsoa_ew.columns
imd_lsoa_ew.columns

In [ ]:
from textwrap import wrap

f, ax = plt.subplots(1, figsize=(20, 20))
ax.set_title('London ambulance service - staff locations and Living Environment deprivation by quintile', fontsize = 15)
plt.tight_layout()
title.set_y(1.01)
imd_lsoa_ew.plot(column='Living Environment Rank (where 1 is most deprived)', scheme='quantiles', k=5, 
         edgecolor='b', cmap=plt.cm.Oranges_r, alpha=1,
         linewidth=0.1, ax=ax, legend=True)
staff_geo_lsoa_ew.plot(ax=ax, marker="x",color="red",markersize=20.0,alpha=1.0)
#tower_hamlets.plot(alpha=1, edgecolor='yellow', linewidth=5.0,ax=ax,facecolor = 'none') - replace with greater London
ax.annotate('Source: London Ambulance Service and London Datastore',
            xy=(1, 0), xycoords='axes fraction',
            xytext=(-20, 20), textcoords='offset pixels',
            horizontalalignment='right',
            verticalalignment='bottom')
plt.tight_layout()
plt.show()
f.savefig('/Users/lucieburgess/Documents/CUSP_data_dive_2019/output/staff_living_env_deprivation.png')


In [ ]:
# Filter data for paramedics, techicians and only and try drawing it again

# Solutions for creating a mask
# https://stackoverflow.com/questions/47781496/python-using-polygons-to-create-a-mask-on-a-given-2d-grid/47813188#47813188

df2 = staff[['PositionTitle','IDnumber']].groupby(['PositionTitle'])['IDnumber'].size().reset_index(name='count').sort_values('count', ascending=False)
print(df2)

# job titles of interest: 
#                                         Paramedic B6   1391
#124                           Emergency Ambulance Crew    645
#282                                       Paramedic NQ    570
#378                 Trainee Emergency Ambulance Crew 2    422
#131                     Emergency Medical Technician 4    317
#128               Emergency Medical Dispatcher Grade 2    202
#70                                Clinical Team Leader    176
#8                                     Ambulance Person    130
#127               Emergency Medical Dispatcher Grade 1    125

In [ ]:
positions_list = ['Paramedic B6','Emergency Ambulance Crew','Paramedic NQ','Trainee Emergency Ambulance Crew 2','Emergency Medical Dispatcher Grade 2','Clinical Team Leader',' Ambulance Person',' Emergency Medical Dispatcher Grade 1']
staff_paramedics = staff_geo_lsoa_ew.loc[staff_geo_lsoa_ew['PositionTitle'].isin(positions_list)]


In [ ]:
staff_paramedics.shape

In [ ]:
staff_geo_lsoa_ew.shape

In [ ]:
staff_paramedics.crs

In [ ]:
staff_paramedics.geometry.name

In [ ]:
# Try the plot again
from textwrap import wrap

f, ax = plt.subplots(1, figsize=(20, 20))
title = ax.set_title("\n".join(wrap('London ambulance service - paramedic home locations and deprivation by quintile', 80)))
plt.tight_layout()
title.set_y(1.01)
imd_lsoa_ew.plot(column='Living Environment Rank (where 1 is most deprived)', scheme='quantiles', k=5, 
         edgecolor='b', cmap=plt.cm.Blues_r, alpha=1,
         linewidth=0.1, ax=ax, legend=True)
staff_paramedics.plot(ax=ax, marker="x",color="red",markersize=20.0,alpha=1.0)
ax.annotate('Source: London Ambulance Service and London Datastore',
            xy=(1, 0), xycoords='axes fraction',
            xytext=(-20, 20), textcoords='offset pixels',
            horizontalalignment='right',
            verticalalignment='bottom')
plt.tight_layout()
plt.show()
f.savefig('/Users/lucieburgess/Documents/KCL/CUSP_data_dive_2019/output/paramedic_living_env_deprivation.png')

In [ ]:
staff_geo_lsoa_ew.columns

In [ ]:
# Calculate max, min, average distance from home to work for each type of staff.

def euclidean_distance(p1, p2):
    return (np.sqrt((p2.x-p1.x)**2 + (p2.y-p1.y)**2))/1000

staff_geo_lsoa_ew['euclidean_distance'] = staff_geo_lsoa_ew.apply(lambda row: euclidean_distance(row['home_coords'],row['work_coords']), axis=1)
staff_geo_lsoa_ew.head()

In [ ]:
# https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
df3 = staff_geo_lsoa_ew.groupby('PositionTitle')['euclidean_distance'].min()
df4 = staff_geo_lsoa_ew.groupby('PositionTitle').agg({'euclidean_distance': [np.min, np.max, np.mean, np.sum]}).sort_values('PositionTitle')


In [ ]:
df4

In [ ]:
# Re-title job roles of staff
def rename_jobtitle(job_title):
    positions_list = ['Paramedic','Technician','Control Assistant','Manager','Clerical Worker','Officer','Paramedic Specialist Practitioner']
    if job_title in positions_list:
        return job_title
    else:
        return 'Other'
    
staff_geo_lsoa_ew['Summary_PositionTitle'] = staff_geo_lsoa_ew.apply(lambda row: rename_jobtitle(row['PositionTitle']), axis=1)
staff_geo_lsoa_ew.head()



In [ ]:
# Exclude outliers. Assume no-one travels more than 100km to get to work on a regular basis
staff_dist_100 = staff_geo_lsoa_ew.loc[staff_geo_lsoa_ew['euclidean_distance'] < 100]

staff_dist_100.shape # (5543, 47)
staff_geo_lsoa_ew.shape # (5751, 47) - excludes approx 200 points
staff_dist_100.plot()

In [ ]:
# Calculate average deprivation of staff compared to London and the UK
# Average deprivation in London

london_imd = pd.read_csv('/Users/lucieburgess/Documents/KCL/Urban_Mind_data/London_IMD_2015/london_imd_2015.csv', header='infer', 
                         dtype={'Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived)': np.float64,
                                'Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs)': np.float64}) 


In [ ]:
london_imd.head()

In [ ]:
df5 = london_imd.groupby('Local Authority District name (2013)').agg({'IMD Rank (where 1 is most deprived)': [np.min, np.max, np.mean]})
df5

In [ ]:
london_imd['IMD Rank (where 1 is most deprived)'].mean() # 14065.32
london_imd['IMD Rank (where 1 is most deprived)'].max() # 32700
london_imd['IMD Rank (where 1 is most deprived)'].min() # 586

In [ ]:
staff_geo_lsoa_ew.columns

In [ ]:
imd.columns

In [ ]:
# Calculate average IMD score of LSOAs where London Ambulance Service staff live - within London and outside
# Join England IMD dataset to the staff_geo_lsoa_ew dataset

staff_geo_lsoa_ew_imd = pd.merge(staff_geo_lsoa_ew, imd, on='lsoa11cd', how='inner')
staff_dist_100 = pd.merge(staff_dist_100, imd, on='lsoa11cd', how='inner')

In [ ]:
staff_dist_100.shape

In [ ]:
# Now filter the imd dataset by the LSOAs in the staff_dist_100 dataframe
# return the lsoas in staff_dist_100 as a list or array

lsoas_100 = staff_dist_100['lsoa11cd'].get_values()
len(lsoas_100)

In [ ]:
imd_100 = imd.loc[imd['lsoa11cd'].isin(lsoas_100)]
imd_100.head()

In [ ]:
# We need to join the imd_100 dataset with the lsoas dataset. The lsoas dataset is a spatial dataset, the 
lsoas_100_sp = lsoas_ew.loc[lsoas_ew['lsoa11cd'].isin(lsoas_100)]
lsoas_100_sp.head()

In [ ]:
type(imd_100)
# Now join this back with the LSOA dataframe - left hand join - to retain the spatial information from the LSOA geodataframe
imd_100.columns
imd_100.shape # 581, 20
imd_100_lsoa = lsoas_100_sp.merge(imd_100, on='lsoa11cd')

In [ ]:
imd_100_lsoa.shape
staff_dist_100.shape

In [ ]:
# Now replot the map:

from textwrap import wrap

f, ax = plt.subplots(1, figsize=(20, 20))
title = ax.set_title("\n".join(wrap('London ambulance service - LAS staff home locations and index of multiple deprivation by quintile, outliers excluded', 150)))
plt.tight_layout()
title.set_y(1.01)
greater_london.plot(alpha=1, edgecolor='orange', linewidth=3.0,ax=ax,facecolor = 'none')
imd_100_lsoa.plot(column='Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived)', scheme='quantiles', k=5, 
    edgecolor='b', cmap=plt.cm.Blues_r, alpha=1,linewidth=0.1, ax=ax, legend=True)
staff_dist_100.plot(ax=ax, marker="x",color="red",markersize=20.0,alpha=1.0)
#tower_hamlets.plot(alpha=1, edgecolor='yellow', linewidth=5.0,ax=ax,facecolor = 'none') - replace with greater London
ax.annotate('Source: London Ambulance Service and London Datastore',
            xy=(1, 0), xycoords='axes fraction',
            xytext=(-20, 20), textcoords='offset pixels',
            horizontalalignment='right',
            verticalalignment='bottom')
plt.tight_layout()
plt.show()
f.savefig('/Users/lucieburgess/Documents/CUSP_data_dive_2019/output/staff_100_deprivation.png')


In [ ]:
# Still difficult to tell what's going on as some of the LSOAs are very small. Overlay London borough boundary
shape_dir = "/Users/lucieburgess/Documents/KCL/Urban_Mind_data/lp-consultation-oct-2009-inner-outer-london-shp/" 
london_shp = gpd.read_file(shape_dir + "lp-consultation-oct-2009-inner-outer-london.shp")
london_shp.head()
london_shp['New_Boundary'] = 'Greater_London'
london_shp.head()
greater_london = london_shp.dissolve(by ='New_Boundary', aggfunc = 'sum')
greater_london.plot()
greater_london.head()


In [ ]:
staff_geo_lsoa_ew.geometry.name

In [ ]:
# Now clip this map to the Greater London boundary online
# Clip imd_lsao_ew and staff_geo_lsoa_ew

import shapely.speedups
shapely.speedups.enable()

greater_london.reset_index(drop=True, inplace=True)
greater_london.head()
imd_mask = imd_lsoa_ew['geometry'].within(greater_london.loc[0, 'geometry'])
imd_masked = imd_lsoa_ew.loc[imd_mask]

In [ ]:
staff_mask = staff_geo_lsoa_ew['home_coords'].within(greater_london.loc[0, 'geometry'])
staff_masked = staff_geo_lsoa_ew.loc[staff_mask]

In [ ]:
f, ax = plt.subplots(1, figsize=(20, 20))
ax.set_title('LAS staff home locations (postcode sector) and Index of Multiple Deprivation by Quintile, outliers excluded', fontsize=15)
plt.tight_layout()
title.set_y(1.01)
greater_london.plot(alpha=1, edgecolor='orange', linewidth=3.0,ax=ax,facecolor = 'none')
imd_masked.plot(column='Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived)', scheme='quantiles', k=5, 
    edgecolor='b', cmap=plt.cm.Purples_r, alpha=1,linewidth=0.1, ax=ax, legend=True)
staff_masked.plot(ax=ax, marker="x",color="red",markersize=30.0,alpha=1.0)
ax.annotate('Source: London Ambulance Service and London Datastore',
            xy=(1, 0), xycoords='axes fraction',
            xytext=(-20, 20), textcoords='offset pixels',
            horizontalalignment='right',
            verticalalignment='bottom')
plt.tight_layout()
plt.show()


In [ ]:
# Calculate average deprivation rank of the IMD score of LAS staff locations
staff_dist_100['Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived)'].mean() # 16,742
# National average rank - 16422

In [ ]:
imd.columns

In [ ]:
df6 = imd.agg({'Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived)': [np.min, np.max, np.mean]})


In [ ]:
df6

In [ ]:
# Compare to the London average
london_imd.columns
df7 = london_imd.agg({'IMD Rank (where 1 is most deprived)': [np.min, np.max, np.mean, np.median]})

In [ ]:
df7

In [ ]:
df8 = staff_dist_100.agg({'Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived)': [np.min, np.max, np.mean, np.median]})
df8

In [ ]:
staff_masked.columns

In [ ]:
imd_staff_masked = staff_masked.merge(imd_100, on='lsoa11cd')

In [ ]:
imd_staff_masked.columns

In [ ]:
df9 = imd_staff_masked.agg({'Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived)': [np.min, np.max, np.mean, np.median]})
df9